[Reference](https://medium.com/better-programming/autoscraper-and-flask-create-an-api-from-any-website-in-less-than-5-minutes-3f0f176fc4a3)

In [1]:
!pip install -U autoscraper flask

Requirement already up-to-date: flask in /usr/local/lib/python3.6/dist-packages (1.1.2)


In [13]:
from autoscraper import AutoScraper


url = 'https://www.ebay.com/sch/i.html?_nkw=iphone'
wanted_list = ['Apple iPhone X 64GB Factory Unlocked Smartphone', 
               '$389.99', 
               'https://www.ebay.com/itm/Apple-iPhone-X-64GB-Factory-Unlocked-Smartphone/254187579586?epid=238944741&hash=item3b2ec2a8c2:g:ZPQAAOSwD6VdpL~9']

scraper = AutoScraper()
result = scraper.build(url=url, wanted_list=wanted_list)

scraper.get_result_similar(url, grouped=True)

{}

In [6]:
# scraper.set_rule_aliases({'rule_0aok': 'title', 'rule_vn5z': 'price', 'rule_buz1': 'url'})

# scraper.keep_rules(['rule_0aok', 'rule_vn5z', 'rule_buz1'])

# scraper.save('ebay-search')

In [5]:
url = 'https://www.etsy.com/search?q=macbook'

wanted_dict = {
    'title': [
        'Apple MacBook Pro i9 32GB 500GB Radeon 560X 15.4 2018 Touch Bar 2.9GHz 6-Core', 
        'Laptop MacBook Premium Ergonomic Wood Stand Holder Computer Gift Nerd Tech Geek Mens, woodworking gift, Home office workspace accessories',
    ],
    'price': ['1,500.00', '126.65'],
    'url': ['851553172']
}

scraper = AutoScraper()
scraper.build(url=url, wanted_dict=wanted_dict)

# get results grouped per rule so we'll know which one to use 
scraper.get_result_similar(url, grouped=True)

{'rule_8u44': ['Italian Leather Tech Envelope Clutch, Macbook Pro Case, iPad Sleeve, Laptop Case, Black, 6 colors, Hand Made in the USA, Buttero Minimal Bag',
  'Laptop MacBook Premium Ergonomic Wood Stand Holder Computer Gift Nerd Tech Geek Mens, woodworking gift, Home office workspace accessories',
  'Juice Box - Mac Apple Logo Cover Laptop Vinyl Decal Sticker Macbook Unique Food Fruit Funny',
  'Real Wood MacBook Case - Ebony wood',
  'Personalized Leather Sleeve for MacBook, MacBook 2020 Sleeve, 13.3 inch Laptop Case, 15 inch Macbook Pro Cover, Leather Laptop Cover',
  'Fashion Laptop Bag Case Messenger Bags for MacBook Air Case Waterproof Notebook Bag Women Men touch bar Sleeve with Belt',
  'Skyrim Inspired We Know vinyl Decal Dark Brotherhood sticker gaming macbook laptop 0193',
  'macbook pro 15 inch sleeve leather Laptop case for 15 inch macbook pro Sleeve for notebook personalized gift for her',
  'Hibiscus Wave Vinyl Sticker, Best Friend Gift, Laptop Decals, Beach Stickers, 

In [7]:
scraper.keep_rules(['rule_705x', 'rule_70m8', 'rule_d9wp', 'rule_kv6p'])

scraper.save('etsy-search')

In [15]:
from autoscraper import AutoScraper
from flask import Flask, request


ebay_scraper = AutoScraper()
etsy_scraper = AutoScraper()
# ebay_scraper.load('ebay-search')
etsy_scraper.load('etsy-search')
app = Flask(__name__)


# def get_ebay_result(search_query):
#     url = 'https://www.ebay.com/sch/i.html?_nkw=%s' % search_query
#     result = ebay_scraper.get_result_similar(url, group_by_alias=True)
#     return _aggregate_result(result)
  
  
def get_etsy_result(search_query):
    url = 'https://www.etsy.com/search?q=%s' % search_query
    result = etsy_scraper.get_result_similar(url, group_by_alias=True)
    result['url'] = [f'https://www.etsy.com/listing/{i}' for i in result['url']]
    return _aggregate_result(result)
  
  
def _aggregate_result(result):
    final_result = []
    for i in range(len(list(result.values())[0])):
        final_result.append({alias: result[alias][i] for alias in result})
    return final_result
  
  
@app.route('/', methods=['GET'])
def search_api():
    query = request.args.get('q')
    return dict(result=get_ebay_result(query) + get_etsy_result(query))
  
  
if __name__ == '__main__':
    app.run(port=8080, host='0.0.0.0')